In [1]:
import pandas as pd
import os
import re
import requests

from utils_kg_search import get_all_observation,get_api_df,get_media_observation,get_media_obs_df
from call_chatgpt_with_zny_asyncio2 import *
from utils_data_format_conversion import *
from utils import preprocess_df
from utils_system_prompt import process_row_and_generate_prompt,generate_sys_prompt
pd.set_option('display.max_colwidth', 1500) #设置为80

### 生成system prompt

In [3]:
data = {
'id':['test-1'],
'turn_id':[1],
'source':['测试'],
'user-query':['世界的第三高峰是哪个'],
'Thought':['涉及事实问答，查询世界第三高峰的信息。'],
'API':['<|api_start|><|kvs|>APINAME=>QASearch<|kve|><|kvs|>CATEGORY=>地理<|kve|><|kvs|>QUERY=>世界第三高峰<|kve|><|kvs|>TAG=>世界&第三高峰<|kve|><|api_end|>'],
'observation':["[['一些内容','一些内容','一些内容']]"],
'assistant':['世界的第三高峰是干城章嘉峰，位于喜马拉雅山脉，海拔高达8586米。'],
'relevant_label':['相关']
}
df = pd.DataFrame(data)

start_date = '2024-01-01'
end_date = '2024-03-31'
# 根据obs发布时间的max-date，生成用户提问时间user_prompt，用户提问时间比max-date随机大1-30
df = df.apply(lambda row: process_row_and_generate_prompt(row, start_date, end_date), axis=1)
# 拼接生成system prompt，地点、时间随机，用户日期根据process_row_and_generate_prompt，如果没有上面一行，自动随机生成一个时间，start_date = "2020-01-01" end_date = "2026-12-31"
df = df.apply(generate_sys_prompt,axis=1)
df

,id,turn_id,source,user-query,Thought,API,observation,assistant,relevant_label,max_date,user_prompt,system
0,test-1,1,测试,世界的第三高峰是哪个,涉及事实问答，查询世界第三高峰的信息。,<|api_start|><|kvs|>APINAME=>QASearch<|kve|><|...,"[['一些内容','一些内容','一些内容']]",世界的第三高峰是干城章嘉峰，位于喜马拉雅山脉，海拔高达8586米。,相关,,2024年03月08日,你是一个名字叫做理想同学的AI数字生命体。\n理想同学是一个可靠的智能家庭助手，由理想汽车智...


### csv格式转为jsonl格式

In [4]:
data = {
'id':['test-1'],
'turn_id':[1],
'source':['测试'],
'user-query':['世界的第三高峰是哪个'],
'Thought':['涉及事实问答，查询世界第三高峰的信息。'],
'API':['<|api_start|><|kvs|>APINAME=>QASearch<|kve|><|kvs|>CATEGORY=>地理<|kve|><|kvs|>QUERY=>世界第三高峰<|kve|><|kvs|>TAG=>世界&第三高峰<|kve|><|api_end|>'],
'observation':["[['一些内容','一些内容','一些内容']]"],
'assistant':['世界的第三高峰是干城章嘉峰，位于喜马拉雅山脉，海拔高达8586米。'],
'relevant_label':['相关']
}
df = pd.DataFrame(data)
sft_df = convert_csv_to_sft(df.copy(),api_flag=True)
sft_df

,id,source,messages
0,test-1,测试,"[{'role': 'user', 'content': ['世界的第三高峰是哪个']}, ..."


### QAsearch

In [5]:
querys = {'user-query':['介绍下刘翔','姚明有多高','世界上最高的山峰是什么？']}
df = pd.DataFrame(querys)

# step 1. 生成API
api_df = get_api_df(df,category='QASearch') # category 具体设置 见函数说明
api_df = api_df.astype(str)

# step 2. 获取observation 
obs_df = get_all_observation(api_df,top_k=5,env='dev') # topk返回obs数量，env环境 dev 或 arch
obs_df

100%|██████████| 3/3 [00:04<00:00,  1.55s/it]


env dev


100%|██████████| 3/3 [00:04<00:00,  1.42s/it]

search_single_api error: 
世界最高的山峰 list index out of range


,user-query,Thought,API,API-NAME,API-CATEGORY,API-QUERY,API-TAG,observation
0,介绍下刘翔,"涉及人物知识, 查询“刘翔的简介”信息作为参考",<|api_start|><|kvs|>APINAME=>QASearch<|kve|><|...,['QASearch'],['人物'],['刘翔的简介'],['刘翔&简介'],[[独家专访刘翔 | 飞过时代的高山大海，飞向他想要的未来 - 知乎 发布时间：2020-1...
1,姚明有多高,涉及人物的身高信息，查询“姚明的身高”信息,<|api_start|><|kvs|>APINAME=>QASearch<|kve|><|...,['QASearch'],['人物'],['姚明的身高是多少'],['姚明&身高'],[[姚明 - 搜狗百科 发布时间：2023-12-19 奥尼尔和姚明合影扮娇小:大姚身高7尺...
2,世界上最高的山峰是什么？,"涉及世界之最知识, 查询“世界最高的山峰”信息作为参考",<|api_start|><|kvs|>APINAME=>QASearch<|kve|><|...,['QASearch'],['世界之最'],['世界最高的山峰'],['世界&最高的山峰'],[[世界十大山峰 世界十大山峰道拉吉里峰道拉吉里峰海拔8172米。所属山脉：喜马拉雅山脉。所...


### AUTOSearch

In [6]:
querys = {'user-query':['理想L9多少钱？','介绍下L8']}
df = pd.DataFrame(querys)

# step 1. 生成API
api_df = get_api_df(df,category='AUTOSearch') # category 具体设置 见函数说明
api_df = api_df.astype(str)

# step 2. 获取observation 
obs_df = get_all_observation(api_df,top_k=3,env='dev') # topk返回obs数量，env环境 dev 或 arch
obs_df

100%|██████████| 2/2 [00:02<00:00,  1.48s/it]


env dev


100%|██████████| 2/2 [00:00<00:00,  5.42it/s]

search_single_api error: 
介绍下L8 list index out of range


,user-query,Thought,API,API-NAME,API-CATEGORY,API-QUERY,API-TAG,observation
0,理想L9多少钱？,涉及汽车问答，需要查询理想L9售价的信息。,<|api_start|><|kvs|>APINAME=>AUTOSearch<|kve|>...,['AUTOSearch'],['汽车'],['理想L9售价'],['理想L9&售价'],[[理想L9介绍 2023年8月3日，理想汽车正式发布理想L9Pro，全国统一零售价42.9...
1,介绍下L8,涉及汽车问答，查询L8的简介信息做参考,<|api_start|><|kvs|>APINAME=>AUTOSearch<|kve|>...,['AUTOSearch'],['汽车'],['L8的简介'],['L8&简介'],[[理想L8简介 尺寸：理想L8传承理想ONE的经典中大型六座SUV的空间布局，车身尺寸为：...


### MEDIASearch

In [7]:
querys = {'user-query':['推荐最近的电影','推荐最新的歌','最近有什么好的电影么？']}
df = pd.DataFrame(querys)

# step 1. 生成API
api_df = get_api_df(df,category='MEDIASearch') # category 具体设置 见函数说明
api_df = api_df.astype(str)

# step 2. 获取observation
obs_df = get_media_obs_df(api_df, top_k=10, env='testtwo') # topk返回obs数量，env环境 testtwo 或 faq
obs_df

100%|██████████| 3/3 [00:05<00:00,  1.74s/it]


,user-query,Thought,API,API-NAME,API-CATEGORY,API-QUERY,API-TAG,slots,observation
0,推荐最近的电影,涉及影视推荐，需要生成影视推荐API完成内容推荐。,<|api_start|><|kvs|>APINAME=>MEDIASearch<|kve|...,['MEDIASearch'],[],['最近的电影'],[],"[{'APINAME': 'MEDIASearch', 'QUERY': '最近的电影', ...",[[《目中无人：以眼还眼》：该影片为2022年网络电影《目中无人》系列的第二部，讲述武艺高强...
1,推荐最新的歌,涉及音乐相关问答，需要生成音乐推荐API完成内容推荐。,<|api_start|><|kvs|>APINAME=>MEDIASearch<|kve|...,['MEDIASearch'],[],['最新的歌'],[],"[{'APINAME': 'MEDIASearch', 'QUERY': '最新的歌', '...",[[《热辣滚烫》：由摩登兄弟刘宇宁演唱，由贾玲、金灿灿作词，钱雷作曲，发行于2024-02-...
2,最近有什么好的电影么？,涉及影视推荐，需要生成影视推荐API完成内容推荐。,<|api_start|><|kvs|>APINAME=>MEDIASearch<|kve|...,['MEDIASearch'],[],['最近有什么好的电影么？'],[],"[{'APINAME': 'MEDIASearch', 'QUERY': '最近有什么好的电...",[[《浩哥闹县衙》：本片讲述被贪官养大的孤儿张浩，为生存不得不学会一身混世本领，阴差阳错间冒...


In [8]:
def get_prompts_df(df: pd.DataFrame, oneshot_prompt: str) -> pd.DataFrame:
    df['prompts'] = df.apply(lambda row: oneshot_prompt + f"""
    ---
    Background:
    {row['observation']}
    Question:
    {row['user-query']}
    Answer：
    """, axis=1)
    
    return df

In [11]:
# step 3. 调用gpt
oneshot_prompt = '根据参考资料回答问题' # 这部分prompt可以根据自己的需求自定义
prompt_df = get_prompts_df(obs_df,oneshot_prompt)
gpt4_df = get_gpt4api_df(prompt_df, max_request_times=5, qps=10, asyncio_flag=False, url = "", temperature=0.5) # 
# max_request_times: 最多重复请求多少次gpt；gpt调用如果有问题，重复调用多少次
# qps: 最大为20，建议设置小于10
# asyncio_flag: 是否使用异步多线程 True = 异步多线程，可在jupyter使用；False = 多线程，可在py文件使用，速度更快
# url：大模型调用API地址

# step 4. 转成jsonl文件
gpt4_df = gpt4_df.astype(str)
gpt4_df['id'] = gpt4_df.index.astype(str)
gpt4_df['source'] = '测试'
sft_df = convert_csv_to_sft(gpt4_df,api_flag=True)
sft_df.to_json('out.jsonl', orient='records', lines=True, force_ascii=False)

gpt4_df

剩余case 3 / 3




  0%|          | 0/3 [00:00<?, ?it/s]

 33%|███▎      | 1/3 [00:37<01:14, 37.31s/it]

 67%|██████▋   | 2/3 [00:44<00:19, 19.86s/it]

100%|██████████| 3/3 [01:06<00:00, 22.08s/it]


,user-query,Thought,API,API-NAME,API-CATEGORY,API-QUERY,API-TAG,slots,observation,assistant,id,source
0,推荐最近的电影,涉及影视推荐，需要生成影视推荐API完成内容推荐。,<|api_start|><|kvs|>APINAME=>MEDIASearch<|kve|...,['MEDIASearch'],[],['最近的电影'],[],"[{'APINAME': 'MEDIASearch', 'QUERY': '最近的电影', ...",[['《目中无人：以眼还眼》：该影片为2022年网络电影《目中无人》系列的第二部，讲述武艺高...,这些电影概述提供了一系列不同类型的故事，从武侠、惊悚、商业阴谋、复仇故事到社会正义等多个主题...,0,测试
1,推荐最新的歌,涉及音乐相关问答，需要生成音乐推荐API完成内容推荐。,<|api_start|><|kvs|>APINAME=>MEDIASearch<|kve|...,['MEDIASearch'],[],['最新的歌'],[],"[{'APINAME': 'MEDIASearch', 'QUERY': '最新的歌', '...",[['《热辣滚烫》：由摩登兄弟刘宇宁演唱，由贾玲、金灿灿作词，钱雷作曲，发行于2024-02...,1. 《热辣滚烫》是由摩登兄弟刘宇宁演唱的国语流行歌曲，贾玲和金灿灿作词，钱雷作曲，发行日期...,1,测试
2,最近有什么好的电影么？,涉及影视推荐，需要生成影视推荐API完成内容推荐。,<|api_start|><|kvs|>APINAME=>MEDIASearch<|kve|...,['MEDIASearch'],[],['最近有什么好的电影么？'],[],"[{'APINAME': 'MEDIASearch', 'QUERY': '最近有什么好的电...",[['《浩哥闹县衙》：本片讲述被贪官养大的孤儿张浩，为生存不得不学会一身混世本领，阴差阳错间...,这段背景资料提供了多部电影的简介。以下是每部电影的概要：\n\n1. 《浩哥闹县衙》：讲述孤...,2,测试
